In [1]:
#Import library
import requests
import pandas as pd
import os
import pymongo
import time
from splinter import Browser
from flask_pymongo import PyMongo
from bs4 import BeautifulSoup 
from flask import Flask,render_template, redirect
#from flask_pymongo import PyMongo

In [2]:
#! pip install chromedriver
#! pip install webdriver-manager

In [3]:
#Create browser instance using splinter

# Setting up windows browser with chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [4]:
#Scraping Mars News
news_url = 'https://redplanetscience.com/'
browser.visit(news_url)
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')


In [5]:
#Collect the latest News Title and Paragraph Text
news_title = news_soup.find_all('div', class_='content_title')[0].text
news_paragraph = news_soup.find_all('div',class_='article_teaser_body')[0].text

print(news_title)
print("_____________________________________________________________")
print(news_paragraph)

Q&A with the Student Who Named Ingenuity, NASA's Mars Helicopter
_____________________________________________________________
As a longtime fan of space exploration, Vaneeza Rupani appreciates the creativity and collaboration involved with trying to fly on another planet.


### JPL Mars Space Images - Featured Image

In [6]:
#Scape Mars Image

jpl_mars_url = 'https://spaceimages-mars.com/'
browser.visit(jpl_mars_url)
html = browser.html
image_soup =  BeautifulSoup(html, 'html.parser')

In [7]:
#Retrive featured image link
featured_image_path = image_soup.find_all('img')[1]['src']
featured_image_url = jpl_mars_url+featured_image_path
print(featured_image_url )

https://spaceimages-mars.com/image/featured/mars3.jpg


### Mars Facts

In [8]:
#Scrape Mars Fact complete table
mars_fact_url = 'https://galaxyfacts-mars.com'

mars_fact_table =pd.read_html(mars_fact_url)
mars_fact_table[0]




,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [9]:
#Extract Mars Fact from the table 
mars_fact_df = mars_fact_table[0]

#Format the header
#grab the first row for the header
new_header = mars_fact_df.iloc[0] 
#take the data less the header row
mars_fact_df = mars_fact_df[1:]
#set the header row as the df header
mars_fact_df.columns = new_header 

#Remove Earth Data
#mars_fact_df =mars_fact_df[['Mars - Earth Comparison', 'Mars']]
#Rename columns
mars_fact_df.rename(columns = {'Mars - Earth Comparison':''}, inplace = True)
mars_fact_df



C:\Users\rchauhan\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [10]:
#Remove ":"
facts = pd.Series(mars_fact_df[''])
mars_fact_df[''] = facts.str.strip(':')
mars_fact_df

<ipython-input-10-b4ad6e3abf08>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mars_fact_df[''] = facts.str.strip(':')


,,Mars,Earth
1,Diameter,"6,779 km","12,742 km"
2,Mass,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons,2,1
4,Distance from Sun,"227,943,824 km","149,598,262 km"
5,Length of Year,687 Earth days,365.24 days
6,Temperature,-87 to -5 °C,-88 to 58°C


In [11]:
#Convert the data to a HTML table string
fact_html_table = mars_fact_df.to_html()
fact_html_table 

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>1</th>\n      <td>Diameter</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Distance from Sun</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Length of Year</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Temperature</td>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

In [12]:
fact_html_table.replace('\n','')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <th>1</th>      <td>Diameter</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>3</th>      <td>Moons</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>4</th>      <td>Distance from Sun</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>5</th>      <td>Length of Year</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>6</th>      <td>Temperature</td>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [13]:
print(fact_html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>Diameter</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Distance from Sun</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Length of Year</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Temperature</td>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


In [14]:
#Save html File
mars_fact_df.to_html('mars_fact_table.html')

### Mars Hemispheres

In [15]:
#Visit the astrogeology site 
hermisphere_url ='https://marshemispheres.com/'
browser.visit(hermisphere_url)
html = browser.html
hemispheres_soup =  BeautifulSoup(html, 'html.parser')

In [16]:
#Retrieve mars hemisphere data

hemispheres_data = hemispheres_soup.find('div', class_ ='collapsible results')
mars_hemispheres = hemispheres_data.find_all('div',class_='item')


In [17]:
#Iterate through each data point to get the title and url
hemisphere_image_urls = []
for i in mars_hemispheres:
    #Retrieve title
    hemisphere =i.find('div', class_="description")
    title= hemisphere.h3.text
    #print(title) --testing
    
    #Retrieve image link by browsing to hemisphere page
    hemisphere_link = hemisphere.a["href"]
    browser.visit(hermisphere_url +hemisphere_link)
    
    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']
    #print (image_url) --testing
    
    #Create dictonary to store title and image_url
    mars_hemisphere_dict = {}
    mars_hemisphere_dict['title'] = title
    mars_hemisphere_dict['img_url'] = hermisphere_url+image_url
    
    hemisphere_image_urls.append(mars_hemisphere_dict)
    
print(hemisphere_image_urls) 

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]
